# Module 5 Project
## Basics
**Jewel Cowan** \
SDS 510 \
November 21, 2025

⭐ Throughout most of this project, I heavily referred to [pandas](https://pandas.pydata.org/pandas-docs/stable/index.html).

In [110]:
from bs4 import BeautifulSoup

from datetime import datetime

from nltk import download, pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from string import punctuation

import json
import math
import os
import pandas as pd
import re

↪ Importing libraries used in the lecture video tutorials this week. \
\
I'm probably importing more than I actually need for this particular project, but I felt it would be safer and save me time in the long run to *have something I don't need*, as opposed to *needing something I don't have* later on.

In [76]:
df = pd.read_json("/content/drive/MyDrive/Fall B/SDS 510: Data Wrangling/Module 5/Projects/Dataset/jeopardy.json")
df

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [77]:
df.index

RangeIndex(start=0, stop=216930, step=1)

In [78]:
df.columns

Index(['category', 'air_date', 'question', 'value', 'answer', 'round',
       'show_number'],
      dtype='object')

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   category     216930 non-null  object
 1   air_date     216930 non-null  object
 2   question     216930 non-null  object
 3   value        213296 non-null  object
 4   answer       216930 non-null  object
 5   round        216930 non-null  object
 6   show_number  216930 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 11.6+ MB


↪ Loading the data and making sure it loaded properly.



---



In [80]:
df[['value', 'question', 'answer', 'category']]

,value,question,answer,category
0,$200,"'For the last 8 years of his life, Galileo was...",Copernicus,HISTORY
1,$200,'No. 2: 1912 Olympian; football star at Carlis...,Jim Thorpe,ESPN's TOP 10 ALL-TIME ATHLETES
2,$200,'The city of Yuma in this state has a record a...,Arizona,EVERYBODY TALKS ABOUT IT...
3,$200,"'In 1963, live on ""The Art Linkletter Show"", t...",McDonald\'s,THE COMPANY LINE
4,$200,"'Signer of the Dec. of Indep., framer of the C...",John Adams,EPITAPHS & TRIBUTES
...,...,...,...,...
216925,$2000,'This Puccini opera turns on the solution to 3...,Turandot,RIDDLE ME THIS
216926,$2000,'In North America this term is properly applie...,a titmouse,"""T"" BIRDS"
216927,$2000,"'In Penny Lane, where this ""Hellraiser"" grew u...",Clive Barker,AUTHORS IN THEIR YOUTH
216928,$2000,"'From Ft. Sill, Okla. he made the plea, Arizon...",Geronimo,QUOTATIONS


↪ Looking at **values** and **questions**, as well as **answers** and **categories**. That may be relevant when considering whether questions are *easy* or *hard*.

In [81]:
df['value'].unique()

array(['$200', '$400', '$600', '$800', '$2,000', '$1000', '$1200',
       '$1600', '$2000', '$3,200', None, '$5,000', '$100', '$300', '$500',
       '$1,000', '$1,500', '$1,200', '$4,800', '$1,800', '$1,100',
       '$2,200', '$3,400', '$3,000', '$4,000', '$1,600', '$6,800',
       '$1,900', '$3,100', '$700', '$1,400', '$2,800', '$8,000', '$6,000',
       '$2,400', '$12,000', '$3,800', '$2,500', '$6,200', '$10,000',
       '$7,000', '$1,492', '$7,400', '$1,300', '$7,200', '$2,600',
       '$3,300', '$5,400', '$4,500', '$2,100', '$900', '$3,600', '$2,127',
       '$367', '$4,400', '$3,500', '$2,900', '$3,900', '$4,100', '$4,600',
       '$10,800', '$2,300', '$5,600', '$1,111', '$8,200', '$5,800',
       '$750', '$7,500', '$1,700', '$9,000', '$6,100', '$1,020', '$4,700',
       '$2,021', '$5,200', '$3,389', '$4,200', '$5', '$2,001', '$1,263',
       '$4,637', '$3,201', '$6,600', '$3,700', '$2,990', '$5,500',
       '$14,000', '$2,700', '$6,400', '$350', '$8,600', '$6,300', '$250',
      

In [82]:
len(df['value'].unique())

150

↪ Looking at how many different value points there are.

In [83]:
df.isnull().sum()

,0
category,0
air_date,0
question,0
value,3634
answer,0
round,0
show_number,0


↪ For full transparency: I searched Kaggle to see if I could find examples of NB classifiers, and found [one](https://www.kaggle.com/code/yash1204/jeopardy-eda-and-random-forest-classifier) that used either the same, or a similar Jeopardy dataset.\
\
This helped me understand how to look for missing points in the set.

In [84]:
df[df['value'] == 'None']

,category,air_date,question,value,answer,round,show_number


↪ That same example on Kaggle used this code and recieved an output table showing all items that had 'None' listed. \
\
However, since this code ran successfully, but didn't actually display anything, I turned to ChatGPT at this point to help me understand why it was working, but *not* working:

In [85]:
df[df['value'].isna()]

,category,air_date,question,value,answer,round,show_number
55,THE SOLAR SYSTEM,2004-12-31,'Objects that pass closer to the sun than Merc...,None,Icarus,Final Jeopardy!,4680
116,HISTORIC WOMEN,2010-07-06,'She was born in Virginia around 1596 & died i...,None,Pocahontas,Final Jeopardy!,5957
174,SPORTS LEGENDS,2000-12-18,'If Joe DiMaggio's hitting streak had gone one...,None,H.J. Heinz (Heinz 57 Varieties),Final Jeopardy!,3751
235,THE MAP OF EUROPE,2000-07-19,"'Bordering Italy, Austria, Hungary & Croatia, ...",None,Slovenia,Final Jeopardy!,3673
296,FAMOUS SHIPS,2006-02-06,"'On December 27, 1831 it departed Plymouth, En...",None,the HMS Beagle,Final Jeopardy!,4931
...,...,...,...,...,...,...,...
216686,MAJOR LEAGUE BASEBALL TEAM NAMES,2001-10-19,'This team received its name after an 1890 inc...,None,Pittsburgh Pirates,Final Jeopardy!,3940
216746,SKYSCRAPERS,2010-12-16,'After a construction boom fueled by oil & gas...,None,Moscow,Final Jeopardy!,6044
216807,NATIONAL CAPITALS,2006-09-29,"'This city's website calls it ""the last divide...",None,Nicosia,Final Jeopardy!,5070
216868,BESTSELLING AUTHORS,2007-03-23,'He had the year's bestselling novel a record ...,None,John Grisham,Final Jeopardy!,5195


In [86]:
newdf = df.dropna(subset=['value'])
newdf

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216924,OFF-BROADWAY,2006-05-11,'In 2006 the cast of this long-running hit emb...,$2000,Stomp,Double Jeopardy!,4999
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999


↪ Now, I could drop the items with no value.

In [87]:
newdf["clean_val"] = newdf["value"].str.replace(r"[^0-9]", "", regex=True).astype(int)
newdf

/tmp/ipython-input-3414657531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf["clean_val"] = newdf["value"].str.replace(r"[^0-9]", "", regex=True).astype(int)


,category,air_date,question,value,answer,round,show_number,clean_val
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680,200
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680,200
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680,200
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680,200
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680,200
...,...,...,...,...,...,...,...,...
216924,OFF-BROADWAY,2006-05-11,'In 2006 the cast of this long-running hit emb...,$2000,Stomp,Double Jeopardy!,4999,2000
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999,2000
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999,2000
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999,2000


In [88]:
newdf = newdf.drop(['air_date', 'round', 'show_number', 'value'], axis=1, errors='ignore')
newdf.head()

,category,question,answer,clean_val
0,HISTORY,"'For the last 8 years of his life, Galileo was...",Copernicus,200
1,ESPN's TOP 10 ALL-TIME ATHLETES,'No. 2: 1912 Olympian; football star at Carlis...,Jim Thorpe,200
2,EVERYBODY TALKS ABOUT IT...,'The city of Yuma in this state has a record a...,Arizona,200
3,THE COMPANY LINE,"'In 1963, live on ""The Art Linkletter Show"", t...",McDonald\'s,200
4,EPITAPHS & TRIBUTES,"'Signer of the Dec. of Indep., framer of the C...",John Adams,200


↪ I watched a [Naive Bayes Classifier](https://www.youtube.com/watch?v=PPeaRc-r1OI&t=341s) tutorial on YouTube and attempted the following code:

```
newdf.drop(['air_date', 'round', 'show_number', 'value'])
newdf.head()
```
That gave me an error that Gemini corrected by adding `axis=1`, but that *also* ran an error, which Gemini corrected again by adding `errors=ignore`.


In [89]:
median = newdf["clean_val"].median()
median

600.0

↪ Looking for the middle number to see *where* to split the values into **high** versus **low**.

In [90]:
newdf["high_low"] = (newdf["clean_val"] >= median).astype(int)
newdf

,category,question,answer,clean_val,high_low
0,HISTORY,"'For the last 8 years of his life, Galileo was...",Copernicus,200,0
1,ESPN's TOP 10 ALL-TIME ATHLETES,'No. 2: 1912 Olympian; football star at Carlis...,Jim Thorpe,200,0
2,EVERYBODY TALKS ABOUT IT...,'The city of Yuma in this state has a record a...,Arizona,200,0
3,THE COMPANY LINE,"'In 1963, live on ""The Art Linkletter Show"", t...",McDonald\'s,200,0
4,EPITAPHS & TRIBUTES,"'Signer of the Dec. of Indep., framer of the C...",John Adams,200,0
...,...,...,...,...,...
216924,OFF-BROADWAY,'In 2006 the cast of this long-running hit emb...,Stomp,2000,1
216925,RIDDLE ME THIS,'This Puccini opera turns on the solution to 3...,Turandot,2000,1
216926,"""T"" BIRDS",'In North America this term is properly applie...,a titmouse,2000,1
216927,AUTHORS IN THEIR YOUTH,"'In Penny Lane, where this ""Hellraiser"" grew u...",Clive Barker,2000,1


↪ I got stuck at this point, trying to figure out *how* to work the median to classify high versus low, so I will credit the responses to [this](https://stackoverflow.com/questions/36679612/classifying-a-numeric-variable-as-high-or-low) Stack Overflow inquiry for reminding me of `>=`.

In [93]:
vect = TfidfVectorizer(max_df=.65, min_df=1, stop_words=None, use_idf=True, norm=None)
transformed = vect.fit_transform(newdf['question'])
transformed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2900153 stored elements and shape (213296, 89903)>

↪ Processed the `'question'` column with the sample code from **Programming Historian**.

In [116]:
X_train, X_test, y_train, y_test = train_test_split(transformed, newdf["high_low"], test_size=.2, random_state=47)

↪ Performed the train/test, mirroring test size and random state numbers from **Gawron**.

In [108]:
len(X_train), len(X_test), len(y_train), len(y_test)

(170636, 42660, 170636, 42660)

In [106]:
X_train

,question
179410,'In 1998 this actress returned to the screen a...
74401,'It's the planetary nickname of Mozart's last ...
167589,"'(<a href=""http://www.j-archive.com/media/2007..."
65751,'I ditched school & never found out that this ...
172107,'1968:<br />Katharine Hepburn'
...,...
25698,'Sheryl Crow in 1999;<br />Guns N' Roses in 1988'
156807,"'Traditionally, a cabinet containing a chamber..."
11723,"'In a 1979 Allan Gurganus bestseller, 99-year-..."
51950,"'Closely identified with Gene Kelly, this song..."


In [109]:
y_train

,high_low
179410,0
74401,1
167589,1
65751,1
172107,1
...,...
25698,1
156807,1
11723,1
51950,0


↪ Checking out of curiosity, since it was done in the same YouTube tutorial I mentioned earlier, which I continued to follow in the next steps:

In [113]:
model = MultinomialNB()

In [117]:
model.fit(X_train, y_train)

MultinomialNB()

In [118]:
model.score(X_test, y_test)

0.5517815283638069

↪ This achieved an accuracy score of about 55%, essentially meaning it's a coin flip as to whether it guessed correctly or not. \
\
If I had to guess, I would charactarize high value questions as hard, and low value questions as easy.